In [4]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import json
from bson import ObjectId  
import folium



## Import DATA FROM MONGO

In [5]:
!mongoimport --host localhost --db mydatabase --collection mycollection --type csv --file D://Project_3_js/file.csv --headerline


2024-03-02T10:18:36.982-0500	connected to: mongodb://localhost/
2024-03-02T10:18:37.046-0500	930 document(s) imported successfully. 0 document(s) failed to import.


In [6]:
# Connect to MongoDB
client = MongoClient('localhost', 27017)  # Assuming MongoDB is running on localhost

# Select the database and collection
db = client['mydatabase']
collection = db['mycollection']


# Find all documents in the collection
cursor = collection.find({})


# Iterate over the documents and print them
for document in cursor:
    print(document)

{'_id': ObjectId('65dd2abea9afe2f6c177c60c'), 'Country Group': 'Emerging Africa', 'Country': 'Kenya', 'indicator code': 'PCPIPCH', 'ISO': 'KEN', 'Subject Descriptor': 'Inflation, average consumer prices', 'Units': 'Percent change', 'Scale': '', 'notes': 'See notes for:  Inflation, average consumer prices (Index).', '2019': 5, '2020': 5, '2021': 6, '2022': 8, 'FY': 2022.0}
{'_id': ObjectId('65dd2abea9afe2f6c177c60d'), 'Country Group': 'Emerging Africa', 'Country': 'Mozambique', 'indicator code': 'PCPIPCH', 'ISO': 'MOZ', 'Subject Descriptor': 'Inflation, average consumer prices', 'Units': 'Percent change', 'Scale': '', 'notes': 'See notes for:  Inflation, average consumer prices (Index).', '2019': 3, '2020': 3, '2021': 6, '2022': 10, 'FY': 2022.0}
{'_id': ObjectId('65dd2abea9afe2f6c177c60e'), 'Country Group': 'Emerging Latin America', 'Country': 'Peru', 'indicator code': 'PCPIPCH', 'ISO': 'PER', 'Subject Descriptor': 'Inflation, average consumer prices', 'Units': 'Percent change', 'Scale

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# review the collections in our database

collections = db.list_collection_names()

collections

['mycollection']

In [8]:
# Get the first document from the cursor
first_document = collection.find_one()

# Extract column names from the first document
column_names = list(first_document.keys())

# Print column names
print(column_names)

['_id', 'Country Group', 'Country', 'indicator code', 'ISO', 'Subject Descriptor', 'Units', 'Scale', 'notes', '2019', '2020', '2021', '2022', 'FY']


In [9]:
# Get all documents from the collection
cursor = collection.find({})

# Convert documents to a list of dictionaries
documents_list = list(cursor)

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(documents_list)

# Print the DataFrame
df

,_id,Country Group,Country,indicator code,ISO,Subject Descriptor,Units,Scale,notes,2019,2020,2021,2022,FY
0,65dd2abea9afe2f6c177c60c,Emerging Africa,Kenya,PCPIPCH,KEN,"Inflation, average consumer prices",Percent change,,"See notes for: Inflation, average consumer pr...",5,5,6,8,2022.0
1,65dd2abea9afe2f6c177c60d,Emerging Africa,Mozambique,PCPIPCH,MOZ,"Inflation, average consumer prices",Percent change,,"See notes for: Inflation, average consumer pr...",3,3,6,10,2022.0
2,65dd2abea9afe2f6c177c60e,Emerging Latin America,Peru,PCPIPCH,PER,"Inflation, average consumer prices",Percent change,,"See notes for: Inflation, average consumer pr...",2,2,4,8,2022.0
3,65dd2abea9afe2f6c177c60f,Emerging Africa,Uganda,PCPIPCH,UGA,"Inflation, average consumer prices",Percent change,,"See notes for: Inflation, average consumer pr...",2,3,2,7,2022.0
4,65dd2abea9afe2f6c177c610,Emerging Latin America,Trinidad and Tobago,NID_NGDP,TTO,Total investment,Percent of GDP,,,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12085,65e3434d424e3c49724e623d,Emerging Asia,Vietnam,NGDPD,VNM,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",332,346,370,406,2022.0
12086,65e3434d424e3c49724e623e,Emerging Asia,Vietnam,PPPEX,VNM,Implied PPP conversion rate,National currency per current international do...,,"See notes for: Gross domestic product, curren...",7530,7542,7418,7200,2022.0
12087,65e3434d424e3c49724e623f,Emerging Asia,Vietnam,PCPIPCH,VNM,"Inflation, average consumer prices",Percent change,,"See notes for: Inflation, average consumer pr...",3,3,2,3,2022.0
12088,65e3434d424e3c49724e6240,Emerging Asia,Vietnam,NID_NGDP,VNM,Total investment,Percent of GDP,,Source: National Statistics Office. Country au...,32,32,33,33,2022.0


## MERGE THE COORDINATES of COUNTRIES IN ONE FILE

In [10]:
# Load the two CSV files into DataFrames
df1 = pd.read_csv('D://Project_3_js/file.csv')
df2 = pd.read_csv('D://Project_3_js/countries.csv')

# Merge the DataFrames based on a common column
merged_df = pd.merge(df1, df2, on='Country')

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

merged_df.to_csv('D://Project_3_js/merged_file.csv', index=False)


merged_df

,Country Group,Country,indicator code,ISO,Subject Descriptor,Units,Scale,notes,2019,2020,2021,2022,FY,latitude,longitude
0,Emerging Latin America,Trinidad and Tobago,NID_NGDP,TTO,Total investment,Percent of GDP,NaN,NaN,0,0,0,0,NaN,10.691803,-61.222503
1,Emerging Latin America,Trinidad and Tobago,NGDP,TTO,"Gross domestic product, current prices",National currency,Billions,Source: National Statistics Office Latest actu...,156,136,162,203,2022.0,10.691803,-61.222503
2,Emerging Latin America,Trinidad and Tobago,NGDPD,TTO,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",23,20,24,30,2022.0,10.691803,-61.222503
3,Emerging Latin America,Trinidad and Tobago,PPPEX,TTO,Implied PPP conversion rate,National currency per current international do...,NaN,"See notes for: Gross domestic product, curren...",4,4,4,5,2022.0,10.691803,-61.222503
4,Emerging Latin America,Trinidad and Tobago,PCPIPCH,TTO,"Inflation, average consumer prices",Percent change,NaN,"See notes for: Inflation, average consumer pr...",1,1,2,6,2022.0,10.691803,-61.222503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Emerging Asia,Vietnam,NGDPD,VNM,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",332,346,370,406,2022.0,14.058324,108.277199
806,Emerging Asia,Vietnam,PPPEX,VNM,Implied PPP conversion rate,National currency per current international do...,NaN,"See notes for: Gross domestic product, curren...",7530,7542,7418,7200,2022.0,14.058324,108.277199
807,Emerging Asia,Vietnam,NID_NGDP,VNM,Total investment,Percent of GDP,NaN,Source: National Statistics Office. Country au...,32,32,33,33,2022.0,14.058324,108.277199
808,Emerging Asia,Vietnam,PCPIPCH,VNM,"Inflation, average consumer prices",Percent change,NaN,"See notes for: Inflation, average consumer pr...",3,3,2,3,2022.0,14.058324,108.277199


## DATA ETL (GENERAL)

In [11]:
# Get the list of columns
columns = merged_df.columns.tolist()

# Find the indices of "ISO" and "indicator code" columns
iso_index = columns.index("ISO")
indicator_code_index = columns.index("indicator code")

# Swap the positions of "ISO" and "indicator code" columns
columns[iso_index], columns[indicator_code_index] = columns[indicator_code_index], columns[iso_index]

# Reindex the DataFrame with the new column order
Clean_df = merged_df.reindex(columns=columns)

# Display the cleaned DataFrame
Clean_df.head()

,Country Group,Country,ISO,indicator code,Subject Descriptor,Units,Scale,notes,2019,2020,2021,2022,FY,latitude,longitude
0,Emerging Latin America,Trinidad and Tobago,TTO,NID_NGDP,Total investment,Percent of GDP,NaN,NaN,0,0,0,0,NaN,10.691803,-61.222503
1,Emerging Latin America,Trinidad and Tobago,TTO,NGDP,"Gross domestic product, current prices",National currency,Billions,Source: National Statistics Office Latest actu...,156,136,162,203,2022.0,10.691803,-61.222503
2,Emerging Latin America,Trinidad and Tobago,TTO,NGDPD,"Gross domestic product, current prices",U.S. dollars,Billions,"See notes for: Gross domestic product, curren...",23,20,24,30,2022.0,10.691803,-61.222503
3,Emerging Latin America,Trinidad and Tobago,TTO,PPPEX,Implied PPP conversion rate,National currency per current international do...,NaN,"See notes for: Gross domestic product, curren...",4,4,4,5,2022.0,10.691803,-61.222503
4,Emerging Latin America,Trinidad and Tobago,TTO,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,"See notes for: Inflation, average consumer pr...",1,1,2,6,2022.0,10.691803,-61.222503


In [13]:
# Drop the '_id' and 'notes' columns
columns_to_drop = ['notes','ISO','Units','Scale']
Final_df = Clean_df.drop(columns=columns_to_drop)

# Print the DataFrame
Final_df

,Country Group,Country,indicator code,Subject Descriptor,2019,2020,2021,2022,FY,latitude,longitude
0,Emerging Latin America,Trinidad and Tobago,NID_NGDP,Total investment,0,0,0,0,NaN,10.691803,-61.222503
1,Emerging Latin America,Trinidad and Tobago,NGDP,"Gross domestic product, current prices",156,136,162,203,2022.0,10.691803,-61.222503
2,Emerging Latin America,Trinidad and Tobago,NGDPD,"Gross domestic product, current prices",23,20,24,30,2022.0,10.691803,-61.222503
3,Emerging Latin America,Trinidad and Tobago,PPPEX,Implied PPP conversion rate,4,4,4,5,2022.0,10.691803,-61.222503
4,Emerging Latin America,Trinidad and Tobago,PCPIPCH,"Inflation, average consumer prices",1,1,2,6,2022.0,10.691803,-61.222503
...,...,...,...,...,...,...,...,...,...,...,...
805,Emerging Asia,Vietnam,NGDPD,"Gross domestic product, current prices",332,346,370,406,2022.0,14.058324,108.277199
806,Emerging Asia,Vietnam,PPPEX,Implied PPP conversion rate,7530,7542,7418,7200,2022.0,14.058324,108.277199
807,Emerging Asia,Vietnam,NID_NGDP,Total investment,32,32,33,33,2022.0,14.058324,108.277199
808,Emerging Asia,Vietnam,PCPIPCH,"Inflation, average consumer prices",3,3,2,3,2022.0,14.058324,108.277199


In [14]:
# Convert 'FY' column to string and extract year
Final_df['FY'] = Final_df['FY'].astype(str).str[:4]

# Print the DataFrame
Final_df

,Country Group,Country,indicator code,Subject Descriptor,2019,2020,2021,2022,FY,latitude,longitude
0,Emerging Latin America,Trinidad and Tobago,NID_NGDP,Total investment,0,0,0,0,nan,10.691803,-61.222503
1,Emerging Latin America,Trinidad and Tobago,NGDP,"Gross domestic product, current prices",156,136,162,203,2022,10.691803,-61.222503
2,Emerging Latin America,Trinidad and Tobago,NGDPD,"Gross domestic product, current prices",23,20,24,30,2022,10.691803,-61.222503
3,Emerging Latin America,Trinidad and Tobago,PPPEX,Implied PPP conversion rate,4,4,4,5,2022,10.691803,-61.222503
4,Emerging Latin America,Trinidad and Tobago,PCPIPCH,"Inflation, average consumer prices",1,1,2,6,2022,10.691803,-61.222503
...,...,...,...,...,...,...,...,...,...,...,...
805,Emerging Asia,Vietnam,NGDPD,"Gross domestic product, current prices",332,346,370,406,2022,14.058324,108.277199
806,Emerging Asia,Vietnam,PPPEX,Implied PPP conversion rate,7530,7542,7418,7200,2022,14.058324,108.277199
807,Emerging Asia,Vietnam,NID_NGDP,Total investment,32,32,33,33,2022,14.058324,108.277199
808,Emerging Asia,Vietnam,PCPIPCH,"Inflation, average consumer prices",3,3,2,3,2022,14.058324,108.277199


In [27]:
Final_df.columns

Index(['Country Group', 'Country', 'ISO', 'indicator code',
       'Subject Descriptor', 'Units', 'Scale', '2019', '2020', '2021', '2022',
       'FY', 'latitude', 'longitude'],
      dtype='object')

## CREATE DATAFRAME by COUNTRY GROUPS

In [28]:
# Filter DataFrame for specified country group
emerging_middle_east_df = Final_df[Final_df['Country Group'] == "Emerging Middle East"]

# Print the result
emerging_middle_east_df

,Country Group,Country,ISO,indicator code,Subject Descriptor,Units,Scale,2019,2020,2021,2022,FY,latitude,longitude
48,Emerging Middle East,Iraq,IRQ,NID_NGDP,Total investment,Percent of GDP,NaN,0,0,0,0,nan,33.223191,43.679291
49,Emerging Middle East,Iraq,IRQ,NGDP,"Gross domestic product, current prices",National currency,Billions,274884,216015,299240,378653,2022,33.223191,43.679291
50,Emerging Middle East,Iraq,IRQ,NGDPD,"Gross domestic product, current prices",U.S. dollars,Billions,233,181,206,261,2022,33.223191,43.679291
51,Emerging Middle East,Iraq,IRQ,PPPEX,Implied PPP conversion rate,National currency per current international do...,NaN,590,520,679,750,2022,33.223191,43.679291
52,Emerging Middle East,Iraq,IRQ,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,0,1,6,5,2022,33.223191,43.679291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,Emerging Middle East,Uzbekistan,UZB,NGDPD,"Gross domestic product, current prices",U.S. dollars,Billions,60,60,70,80,2022,41.377491,64.585262
686,Emerging Middle East,Uzbekistan,UZB,PPPEX,Implied PPP conversion rate,National currency per current international do...,NaN,2056,2261,2457,2614,2022,41.377491,64.585262
687,Emerging Middle East,Uzbekistan,UZB,NID_NGDP,Total investment,Percent of GDP,NaN,40,38,40,38,2022,41.377491,64.585262
688,Emerging Middle East,Uzbekistan,UZB,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,15,13,11,11,2022,41.377491,64.585262


In [32]:
inflation_df = emerging_middle_east_df[emerging_middle_east_df['Subject Descriptor'] == 'Inflation, average consumer prices']
inflation_df

,Country Group,Country,ISO,indicator code,Subject Descriptor,Units,Scale,2019,2020,2021,2022,FY,latitude,longitude
52,Emerging Middle East,Iraq,IRQ,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,0,1,6,5,2022,33.223191,43.679291
58,Emerging Middle East,Somalia,SOM,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,5,4,5,7,2022,5.152149,46.199616
410,Emerging Middle East,Algeria,DZA,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,2,2,7,9,2022,28.033886,1.659626
532,Emerging Middle East,Afghanistan,AFG,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,2,6,5,14,2022,33.939110,67.709953
534,Emerging Middle East,Bahrain,BHR,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,1,-2,-1,4,2022,25.930414,50.637772
544,Emerging Middle East,Armenia,ARM,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,1,1,7,9,2022,40.069099,45.038189
550,Emerging Middle East,Azerbaijan,AZE,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,3,3,7,14,2022,40.143105,47.576927
556,Emerging Middle East,Djibouti,DJI,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,3,2,1,5,2022,11.825138,42.590275
562,Emerging Middle East,Egypt,EGY,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,14,6,5,9,2022,26.820553,30.802498
568,Emerging Middle East,Jordan,JOR,PCPIPCH,"Inflation, average consumer prices",Percent change,NaN,1,0,1,4,2021,30.585164,36.238414


## CHANGE DATA TYPE TO GEOJASON

In [13]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# Create Point geometries from latitude and longitude columns
geometry = [Point(xy) for xy in zip(emerging_middle_east_df['longitude'], emerging_middle_east_df['latitude'])]

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(emerging_middle_east_df, geometry=geometry)

# Convert GeoDataFrame to GeoJSON
geojson = gdf.to_json()
output_path = 'D://Project_3_js/emerging_middle_east.geojson'

with open(output_path, 'w') as file:
    file.write(geojson)

with open('emerging_middle_east.geojson', 'w') as file:
    file.write(geojson) 
    
# Print or save the GeoJSON data
geojson



'{"type": "FeatureCollection", "features": [{"id": "48", "type": "Feature", "properties": {"2019": 0, "2020": 0, "2021": 0, "2022": 0, "Country": "Iraq", "Country Group": "Emerging Middle East", "FY": "nan", "ISO": "IRQ", "Scale": null, "Subject Descriptor": "Total investment", "Units": "Percent of GDP", "indicator code": "NID_NGDP", "latitude": 33.223191, "longitude": 43.679291}, "geometry": {"type": "Point", "coordinates": [43.679291, 33.223191]}}, {"id": "49", "type": "Feature", "properties": {"2019": 274884, "2020": 216015, "2021": 299240, "2022": 378653, "Country": "Iraq", "Country Group": "Emerging Middle East", "FY": "2022", "ISO": "IRQ", "Scale": "Billions", "Subject Descriptor": "Gross domestic product, current prices", "Units": "National currency", "indicator code": "NGDP", "latitude": 33.223191, "longitude": 43.679291}, "geometry": {"type": "Point", "coordinates": [43.679291, 33.223191]}}, {"id": "50", "type": "Feature", "properties": {"2019": 233, "2020": 181, "2021": 206, 

In [34]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


# Create Point geometries from latitude and longitude columns
geometry = [Point(xy) for xy in zip(inflation_df['longitude'], inflation_df['latitude'])]

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(inflation_df, geometry=geometry)

# Convert GeoDataFrame to GeoJSON
geojson = gdf.to_json()
output_path = 'D://Project_3_js/inflation.geojson'

with open(output_path, 'w') as file:
    file.write(geojson)

with open('inflation.geojson', 'w') as file:
    file.write(geojson) 
    
# Print or save the GeoJSON data
geojson



'{"type": "FeatureCollection", "features": [{"id": "52", "type": "Feature", "properties": {"2019": 0, "2020": 1, "2021": 6, "2022": 5, "Country": "Iraq", "Country Group": "Emerging Middle East", "FY": "2022", "ISO": "IRQ", "Scale": null, "Subject Descriptor": "Inflation, average consumer prices", "Units": "Percent change", "indicator code": "PCPIPCH", "latitude": 33.223191, "longitude": 43.679291}, "geometry": {"type": "Point", "coordinates": [43.679291, 33.223191]}}, {"id": "58", "type": "Feature", "properties": {"2019": 5, "2020": 4, "2021": 5, "2022": 7, "Country": "Somalia", "Country Group": "Emerging Middle East", "FY": "2022", "ISO": "SOM", "Scale": null, "Subject Descriptor": "Inflation, average consumer prices", "Units": "Percent change", "indicator code": "PCPIPCH", "latitude": 5.152149, "longitude": 46.199616}, "geometry": {"type": "Point", "coordinates": [46.199616, 5.152149]}}, {"id": "410", "type": "Feature", "properties": {"2019": 2, "2020": 2, "2021": 7, "2022": 9, "Coun

In [15]:


# Load the GeoJSON data
with open('D://Project_3_js/emerging_middle_east.geojson') as f:
    geojson_data = json.load(f)

# Update the GeoJSON URL in the HTML template
html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Leaflet Map</title>
    <!-- Add Leaflet CSS -->
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <!-- Add Leaflet JavaScript -->
    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <!-- Add JQuery -->
    <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
</head>
<body>
    <!-- Your map container -->
    <div id="map" style="height: 600px;"></div>

    <script>
        // Function to create map
        function createMap() {
            // Initialize the map
            var map = L.map('map').setView([0, 0], 2);

            // Add a tile layer from OpenStreetMap
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                attribution: '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'
            }).addTo(map);

            // Load GeoJSON data
            var geojson_data = """ + json.dumps(geojson_data) + """;

            // Add GeoJSON layer to the map
            L.geoJSON(geojson_data).addTo(map);
        }

        // Call the createMap function when the page is loaded
        $(document).ready(function() {
            createMap();
        });
    </script>
</body>
</html>
"""

# Save the HTML content to a new file
with open('leaflet_map.html', 'w') as f:
    f.write(html_content)

In [22]:
import plotly.graph_objects as go
# Create a base map
fig = go.Figure()
# Iterate through each row in the DataFrame
for index, row in Final_df.iterrows():
    # Extract relevant information
    country = row['Country']
    latitude = row['latitude']
    longitude = row['longitude']
    # Add a scatter point for each country
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = [longitude],
        lat = [latitude],
        text = country,
        mode = 'markers',
        marker = dict(
            size = 10,
            opacity = 0.8,
            symbol = 'circle',
            line = dict(width=0)
        ),
        name = country
    ))
# Update the layout to set the initial map center and zoom
fig.update_geos(projection_type="orthographic",
                showcoastlines=True,
                coastlinecolor="RebeccaPurple",
                showland=True, landcolor="LightGreen",
                showocean=True, oceancolor="Azure",
                showlakes=True, lakecolor="Azure")
# Save the map to an HTML file
fig.write_html("map_index_plotly.html", auto_open=True)







